# DREAM data reduction

We begin with relevant imports.
We will be using tutorial data downloaded with `pooch`.
If you get an error about a missing module `pooch`, you can install it with `!pip install pooch`:

In [ ]:
import scipp as sc
from scippneutron.io import cif

from ess import dream, powder
import ess.dream.data  # noqa: F401
from ess.powder.types import *

## Create and configure the workflow

We begin by creating the Dream (Geant4) workflow object which is a skeleton for reducing Dream data, with pre-configured steps.

In [ ]:
workflow = dream.DreamGeant4Workflow()

We then need to set the missing parameters which are specific to each experiment
(the keys are types defined in [essdiffraction.powder.types](../generated/modules/ess.diffraction.powder.types.rst)):

In [ ]:
workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None
workflow[NeXusDetectorName] = "mantle"
# The upper bounds mode is not yet implemented.
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
# Edges for binning in d-spacing
workflow[DspacingBins] = sc.linspace("dspacing", 0.0, 2.3434, 201, unit="angstrom")
# Mask in time-of-flight to crop to valid range
workflow[TofMask] = lambda x: (x < sc.scalar(0.0, unit="ns")) | (
    x > sc.scalar(86e6, unit="ns")
)
workflow[TwoThetaMask] = None
workflow[WavelengthMask] = None
# No pixel masks
workflow = powder.with_pixel_mask_filenames(workflow, [])

We also need some parameters to configure the output file:

In [ ]:
workflow[CIFAuthors] = CIFAuthors([
    cif.Author(
        name="Jane Doe",
        email="jane.doe@ess.eu",
        orcid="0000-0000-0000-0001",
        role="measurement",
    ),
])

## Use the workflow

We can visualize the graph for computing the final normalized result for intensity as a function of d-spacing:

In [ ]:
workflow.visualize(IofDspacing, graph_attr={"rankdir": "LR"})

We then call `compute()` to compute the result:
(The `cif` object will later be used to write the result to disk.)

In [ ]:
results = workflow.compute([IofDspacing, ReducedDspacingCIF])
result = results[IofDspacing]
cif_data = results[ReducedDspacingCIF]

In [ ]:
dspacing_histogram = result.hist()
dspacing_histogram.plot()

We can now save the result to disk:
(The comment is optional but helps to identify the file later.)

In [ ]:
cif_data.comment = """This file was generated with the DREAM data reduction user guide
in the documentation of ESSdiffraction.
See https://scipp.github.io/essdiffraction/
"""
cif_data.save('dspacing.cif')

## Compute intermediate results

For inspection and debugging purposes, we can also compute intermediate results.
To avoid repeated computation (including costly loading of files), we can request multiple results at once, including the final result, if desired.
For example:

In [ ]:
intermediates = workflow.compute(
    (
        DataWithScatteringCoordinates[SampleRun],
        MaskedData[SampleRun],
    )
)

intermediates[DataWithScatteringCoordinates[SampleRun]]

In [ ]:
two_theta = sc.linspace("two_theta", 0.8, 2.4, 301, unit="rad")
intermediates[MaskedData[SampleRun]].hist(two_theta=two_theta, wavelength=300).plot(
    norm="log"
)

## Grouping by scattering angle

The above workflow focuses the data by merging all instrument pixels to produce a 1d d-spacing curve.
If instead we want to group into $2\theta$ bins, we can alter the workflow parameters by adding some binning in $2\theta$:

In [ ]:
workflow[TwoThetaBins] = sc.linspace(
    dim="two_theta", unit="rad", start=0.8, stop=2.4, num=17
)

In [ ]:
grouped_dspacing = workflow.compute(IofDspacingTwoTheta)
grouped_dspacing

In [ ]:
angle = sc.midpoints(grouped_dspacing.coords["two_theta"])
sc.plot(
    {
        f"{angle[group].value:.3f} {angle[group].unit}": grouped_dspacing[
            "two_theta", group
        ].hist()
        for group in range(2, 6)
    }
)

In [ ]:
grouped_dspacing.hist().plot(norm="log")